In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
import sys
sys.path.append("/home/ray__/ssd/BERT/")
sys.path.append("/home/ray__/CS/org/etherlabs/ai-engine/pkg/")
sys.path.append("../")
from gpt_feat_utils import GPT_Inference

gpt_model = GPT_Inference("/home/ray__/ssd/BERT/models/se/epoch3/", device="cpu")
#gpt_model = GPT_Inference("/home/ray__/ssd/BERT/models/ether/", device="cpu")

# with open('../topic_testing/cullen_test.json','rb') as f:
#     request = json.load(f)
#     if isinstance(request, str):
#         request = json.loads(request)


ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [17]:
import pickle
import numpy as np
import networkx as nx
# S.E
ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/se/se_entity_feats_v3.pkl","rb"))
sent_dict = pickle.load(open("/home/ray__/ssd/minds/se/se_sent_dict_v3.pkl", "rb"))
com_map = pickle.load(open("/home/ray__/ssd/minds/se/tests/com_map.pkl", "rb"))
# ent_graph = nx.read_gpickle("/home/ray__/ssd/minds/se/se_ent_graph_wcosine_pruned_v3.gpkl")

# Ether
# ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/ether/ether_entity_feats_grp.pkl","rb"))
# sent_dict = pickle.load(open("/home/ray__/ssd/minds/ether/ether_sent_dict_v1.pkl", "rb"))
# com_map = pickle.load(open("/home/ray__/ssd/minds/ether_se/com_map.pkl", "rb"))

# Ether_se

# ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/ether_se/se_ether_feats_non_updated_allether.pkl", "rb"))
# com_map = pickle.load(open("/home/ray__/ssd/minds/ether_se/com_map_h.pkl", "rb"))
# sent_dict = pickle.load(open("/home/ray__/ssd/minds/ether/ether_single_label_dict_v1.pkl","rb"))

# # A.Ient
# ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/ai/ai_entity.pkl", "rb"))
# com_map = pickle.load(open("/home/ray__/ssd/minds/ai/com_map_ai.pkl", "rb"))
# ent_graph = nx.read_gpickle("/home/ray__/ssd/minds/ai/ai_pruned_entity_wfv.gpkl")


# common_entities = ent_fv_full.keys() & com_map.keys()
# ent_fv = {}
# for ent in common_entities:
#     ent_fv[ent] = ent_fv_full[ent]

common_entities = ent_fv_full.keys() & com_map.keys()
ent_fv = {}
for ent in common_entities:
    if ent in sent_dict.keys() and sent_dict[ent]=='PER':
        continue
    ent_fv[ent] = ent_fv_full[ent]

In [18]:
import text_preprocessing.preprocess as tp
from extra_preprocess import preprocess_text
import numpy as np
from scipy.spatial.distance import cosine

def generate_gs(set_list):
    com_freq = {}
    for file in set_list:
        req = json.load(open(file, "r"))
        if isinstance(req, str):
            request = json.loads(req)["body"]
        else:
            request = req["body"]
            
        request["segments"] = sorted(request['segments'], key=lambda kv:kv['startTime'])
        for index, seg in enumerate(request["segments"]):
            request["segments"][index]["originalText"] = " ".join(preprocess_text(seg["originalText"]))
        segments_map = {}
        for index, seg in enumerate(request["segments"]):
            if seg["originalText"] != "":
                segments_map[seg['id']] = seg
                # if len(seg["originalText"].split(". "))==1 and len(seg["originalText"].split(" "))<=6 :
                #continue
                segments_map[seg['id']]["order"] = index
        text = list(map(lambda seg: (seg["originalText"], seg["id"]), [segment for segment in request['segments'] if segment["originalText"]!=""]))
        seg_list = [sent for sent, id  in text]
        segid_list = [id for sent, id in text]
        sent_list = list(map(lambda seg, segid:([sent + ". " for sent in seg.split(". ")],segid), seg_list, segid_list))
        sent_list = [(sent, segid) for seg, segid in sent_list for sent in seg]
        
        segments_fv = {}
        for segments in segments_map.values():
            mod_sent = preprocess_text(segments["originalText"])
            if mod_sent:
                sent_fv = list(map(lambda kv: gpt_model.get_text_feats(kv), mod_sent))
                segments_fv[segments["id"]] = np.mean(sent_fv, axis=0)
        
        
        ent_score = {}
        for segid in segments_fv.keys():
            ent_score[segid] = []
            for ent in ent_fv.keys():
                ent_score[segid].append( (ent, 1 - cosine(segments_fv[segid], ent_fv[ent])))
                
        keys_in_ent_score = ent_score.keys()
        ent_score_sorted = {}
        for segid in keys_in_ent_score:
            sorted_score = sorted(ent_score[segid], key=lambda x: x[1], reverse=True)
            ent_score_sorted[segid] = [i for i,j in sorted_score][:10]
            for ent in [i for i,j in sorted_score][:10]:
                if com_map[ent] in com_freq.keys():
                    com_freq[com_map[ent]] += 1
                else:
                    com_freq[com_map[ent]] = 1
            print ("\n\n\n", segments_map[segid]["originalText"], "\n\n Entities picked: ", [i for i,j in sorted_score][:10], "\n\n community mapped: ",  [com_map[i] for i in [i for i,j in sorted_score][:10]])
        
    return com_freq

In [8]:
import os
dataset = os.listdir("../golden_dataset/ether/")

ranked = generate_gs(["../golden_dataset/ether/"+i for i in dataset if i!=".ipynb_checkpoints"])




 I just done with some pocs one thing which I wanted to figure it out like why it was happening. I frame and why it is not in the with the web and if I hosted our entire lives, it is a mate on the iframe whether I could able to hear the sounds or not. 

 Entities picked:  ['Gypsy', 'Janus', 'Hermes', 'Internet', 'Dixie', 'Call', 'Psi', 'Pimm', 'Link', 'Web Sdk'] 

 community mapped:  [91, 91, 91, 77, 91, 77, 91, 91, 77, 137]



 The first one like when I hosted it on iframe, I could able to get the audio without any distortion. But one more effort we had was the mixing and then I was doing the mixing yesterday. I was booking on the mixing part in that like we do not have to do much work by because it seemed it always gives the updated strength initially only once we have to do it because since I was Computing again, and again, it was giving me Cool. So right now like I have a I can get the audio output which like after all of our conversation. And also I get the transcript the probl




 So after the coral test XnumberX and XnumberX and Q and do some basic right corner one or two tests on that if that works fine. So I mean one thing that is pending from my series like I need to test any to a duties for that for my implementation. Ask I am going to say you tease me for my boots. 

 Entities picked:  ['Gypsy', 'Code', 'Xyz', 'Porsche', 'Qa', 'Janus', 'Yay', 'Ics', 'Em', 'Cat'] 

 community mapped:  [91, 72, 72, 77, 44, 91, 72, 41, 41, 72]



 I hope this trip years ago during that maybe binder today are on Thursday can do a test on this one also. 

 Entities picked:  ['Beijing', 'Hope', 'Life Unit', 'Gothic', 'Topgel', 'Valid', 'Ether Data', 'Shawshank', 'Doc', 'Pulseaudio'] 

 community mapped:  [175, 43, 133, 90, 175, 219, 217, 91, 114, 272]



 Okay, I will let us around you will just do the same similar thing except if the providers also Thailand, right? 

 Entities picked:  ['Ether Labs', 'Ether Meet', 'Janus', 'Apa', 'Ah', 'Sigmoid', 'Delorean', 'China', 'Psi',

 community mapped:  [91, 91, 91, 91, 137, 137, 77, 91, 91, 77]



 May I speak Society discard draft changes are in staging right now. So what we can do is I will create release today and we can release these changes tomorrow in beta products with the router changes. I like I have tested the router changes thoroughly will test. Again, I will sit with me clean and a dentist again. I will create a doc and share in the channel and we will test again thoroughly with Franklin and like little later released tomorrow in the morning, beautiful. 

 Entities picked:  ['Dixie', 'Alberta', 'Hermes', 'Porsche', 'Gypsy', 'Website Staging', 'Production', 'Drawing Board', 'Janus', 'National'] 

 community mapped:  [91, 114, 91, 77, 91, 175, 114, 44, 91, 77]



 But okay, so let us take let us take let us kill that cat first. So let us say we want to create publish this card same draft feature completely into production. A flexible side is all the innocent platform already in production, and it is only




 I think I have a couple of tasks to Meanwhile like the puzzle game and then removing the polling and using web socket and then I can 

 Entities picked:  ['Web Sdk', 'Fire Apis', 'Url Web', 'Web', 'Dixie', 'Gypsy', 'Web Api', 'List Api', 'Accessory View', 'List View Api'] 

 community mapped:  [137, 205, 327, 137, 91, 91, 77, 313, 312, 311]



 Okay, he for iOS and my people if you going to say you will need to decide if it is easy for you to continue on one branch, which is another router changes make these changes for some production. But if you think they are already very close it all the router changes and if it will help accelerate the both of your working together. 

 Entities picked:  ['Dixie', 'Shawshank', 'Janus', 'Tiger', 'Hermes', 'Lander', 'Production', 'Stats', 'Psi', 'Gypsy'] 

 community mapped:  [91, 91, 91, 72, 91, 16, 114, 99, 91, 91]



 Yes, I I have added question detection in staging but I am getting some errors. So I am trying to figure out trying to solve th




 Yeah, we can do it because right now and he will not because no one else is that this is purely what do you say disaster scenario insurance, right? 

 Entities picked:  ['Wait', 'Lotus', 'Gosh', 'Cappuccino', 'Hell', 'Well', 'Janus', 'Ugh', 'Martian', 'Know'] 

 community mapped:  [72, 74, 72, 42, 42, 72, 91, 72, 72, 42]



 Janus is purely at because we have no plans to work with jealous any effort on it or anything only. We only reason we keep it is because if for whatever reason jitsi is goes down. 

 Entities picked:  ['Jitsi', 'Janus', 'Psi', 'Shawshank', 'Pimm', 'Neptune', 'Space', 'Titanic', 'Happo', 'Dixie'] 

 community mapped:  [83, 91, 91, 91, 91, 24, 72, 72, 73, 91]



 With some effort we can move bring the arms back up. It may not again when I was as soon as lot of other things that will have to do then we can do that. The other option is to see you know, we would have used to discuss your strategy. 

 Entities picked:  ['Em', 'Board', 'Pimm', 'Psi', 'Gypsy', 'Drawing

...




 Uhhuh for the first call we go with the main domain with as I said every mind would be associated with the key phrase graph the default Keepsake. That is the domain key phrase graph that means for now. 

 Entities picked:  ['Graph', 'Domain', 'Entity', 'Id', 'Film', 'Technologies', 'Title', 'Context Id', 'Data Model', 'Identifier'] 

 community mapped:  [23, 80, 144, 80, 80, 148, 80, 18, 73, 80]



 So for the First Column we will do a mapping of the key phrase with whatever if this is they have been able to exact from the Mind from the model. 

 Entities picked:  ['Classifier', 'Cosine', 'Wine', 'Fact', 'Dictionary', 'Rapidminer', 'Score', 'Ema', 'Order', 'Dimension'] 

 community mapped:  [147, 147, 147, 18, 148, 19, 147, 143, 148, 18]



 Yeah, that will be a little unfair because we will not know we might not know lot of key phrases from the first call. So we will do that first and then I am going to figure out what we can do to reduce that impact of aggressive filtration. 

 E




 The most of the modal dialog is small we can do because we have like a bunch of chemicals, right? So even if it I understand the conflict the the commission issue, but what is like given so you want me to assist the number but something simple some like can we what are what will happen? For example, if he appointed date like today is college like XnumberX Jan XnumberX XnumberX XnumberX years something so I know II just did XnumberX. I know that I need to also put the same name and the same day. I can put the time also, that is okay then it is like a number less. My point is why do not you just take a chance and just from the date? Hope you do not know that you  repeating same data. You can collect multiple meetings same at the same time is when we it does not include the time. 

 Entities picked:  ['E', 'Isn', 'Uuid', 'D', 'Pentagon', 'Janus', 'Uuids', 'X', 'Call', 'Utc'] 

 community mapped:  [72, 74, 80, 74, 72, 91, 80, 72, 77, 88]



 Got if you just say XnumberXth June this thi




 It should not be because if you see currently Only The Ether is ether models are manually enter VDP. If you bring that to the same as domain models, which are dependent on into the graphs. 

 Entities picked:  ['Domain', 'D', 'Hexarch', 'Graph', 'Ether', 'X', 'V', 'Global', 'Ids', 'B'] 

 community mapped:  [80, 74, 144, 23, 72, 72, 74, 74, 80, 74]



 It is actually even currently we just do it manually, but but when we have this entity graph fully then we ideally there should not be any human intervention because it gets updated after every call without any periodic updates. 

 Entities picked:  ['Actions Teams', 'Ormlite', 'State', 'Es', 'Geo', 'Udf', 'Upstream', 'Database', 'Blog Post', 'Entity'] 

 community mapped:  [16, 24, 16, 24, 24, 80, 78, 74, 75, 144]



 We just get to populate it after every call, but but eventually what happens is I mean the we are not involved even populations if you get populated after every week, every transcription actually got it. 

 Entities pi




 Third look if I have tried to calls like after deployment like I got the transcript completely from the calls. Mmm, and now like I will just check whether it is my back but I am short like it. So let us string error from the master Branch as well. Okay, so that is the only known issue it but that perspective. 

 Entities picked:  ['Hermes', 'Dixie', 'Janus', 'Gypsy', 'Call', 'Porsche', 'Delorean', 'Console', 'Status', 'Aids 800189001'] 

 community mapped:  [91, 91, 91, 91, 77, 77, 91, 77, 77, 199]



 Here is what I want you to do Franklin just to validate it. I want you to play the same file in production and in staging. Dont wait every large meeting do it for a small meeting. Okay about like one or two minutes of audio playback, OK and then is what you see in production and what you see in staging from a transcript perspective. 

 Entities picked:  ['Janus', 'Production', 'Gypsy', 'Cat', 'Porsche', 'Dixie', 'Gosh', 'Hermes', 'Link', 'Fyi'] 

 community mapped:  [91, 114, 91, 72,




 Because there is so many other things in the product we have to do and I want to spend like another week or two and the website is right. So I want to get it to a reasonable stability where at least XnumberX to XnumberX simultaneous web used page views and work and everything will work. So in that case, let us see what how we are doing with the current memory size and then we even still running into issues will go to college. So I started with with XnumberX then XnumberX XnumberX also requests per minute that means 

 Entities picked:  ['Gypsy', 'Janus', 'Pimm', 'Max', 'Psi', 'Land', 'Dixie', 'Shawshank', 'Iphone', 'Iphones'] 

 community mapped:  [91, 91, 91, 72, 91, 72, 91, 91, 77, 77]



 You guys say so even this and instead of like putting time into this if we run into issues to because these issues basically came when analytics was showing around XnumberX views and that time it was XnumberX and Beyond lie. 

 Entities picked:  ['Psy', 'Janus', 'Pimm', 'Gypsy', 'Graham', 'Shaw

In [9]:
ranked_sorted = sorted(ranked.items(), key=lambda kv:kv[1], reverse=True)

clusters = {}
temp = []
prev_com = 0
for index,(word,cluster) in enumerate(sorted(com_map.items(), key=lambda kv:kv[1])):
    if prev_com==cluster:
        temp.append(word)
        if index==len(com_map.items())-1:
            clusters[cluster] = temp
    else:
        clusters[prev_com] = temp
        temp = []
        prev_com = cluster
        temp.append(word)
        if index==len(com_map.items())-1:
            clusters[cluster] = temp

In [10]:
for com, freq in ranked_sorted:
    print ("\n\nCommunity: ", com, "  Freq: ", freq ,"\n")
    print (clusters[com])



Community:  91   Freq:  712 

['Colin', 'Gypsy', 'Hermes', 'Psi', 'Delorean', 'Pimm', 'Cullen', 'Shank', 'Dixie', 'Collins', 'Shawshank', 'Janus']


Community:  77   Freq:  348 

['Anna', 'Applications', 'Start', 'Ether Labs', 'Used', 'Call', 'Ok', 'Gypsies', 'Jen', 'Backend', 'Webapps', 'Application', 'Api', 'Cfctl', 'App', 'Skype', 'Janice', 'Web Ui', 'Path', 'Location', 'Netflix', 'Pasha', 'Nach', 'Next', 'Webapi', 'Epa', 'Allah', 'Imap', 'Webserver', 'Tmp', 'Status', 'Porsche', 'Slack', 'Something', 'Server', 'Uids', 'Playlist', 'Internet', 'Js App', 'Iphones', 'Tinyurl', 'Eddie', 'Region', 'Service', 'Apm', 'Web-application', 'Router', 'Iphone', 'Sdk', 'Apa', 'Services', 'Link', 'Web App', 'Web Api', 'Http Api', 'Webapp', 'Gotchas', 'Connect', 'Sigmoid', 'Site', 'Box', 'Cf', 'Post', 'Api Docs', 'Epa Gateway', 'App Engine', 'Iis', 'Aps', 'Webservers', 'Apis', 'Epa Google Apis', 'Apps', 'Recorder', 'Cdk', 'Ether Client Ether Meet', 'Pong', 'Pool', '__url__', 'Website', 'Console', 

In [11]:
com_map_selected = {}
for index, (com, freq) in enumerate(ranked_sorted):
    com_map_selected[com] = index

In [12]:
pickle.dump(ranked, open("/home/ray__/ssd/minds/ether_se/gc.pkl", "wb"))

In [13]:
pickle.dump(com_map_selected, open("/home/ray__/ssd/minds/ether_se/ranked_com_selected_h.pkl", "wb"))

In [14]:
lc = {}
for cls, freq in ranked.items():
    lc[cls] = []

In [15]:
pickle.dump(lc, open("/home/ray__/ssd/minds/ether_se/lc.pkl", "wb"))

In [41]:
clusters[5]

['Web',
 'Google',
 'Ios',
 'Janus',
 'Zoom',
 'Gypsy',
 'App',
 'Internet',
 'App Store',
 'Ios App']

In [2]:
import pickle
com_map = pickle.load(open("/home/ray__/ssd/minds/ether/com_map.pkl", "rb"))

In [3]:
ranked_sorted = sorted(com_map.items(), key=lambda kv:kv[1], reverse=True)

clusters = []
temp = []
prev_com = 0
for index,(word,cluster) in enumerate(sorted(com_map.items(), key=lambda kv:kv[1])):
    if prev_com==cluster:
        temp.append(word)
        if index==len(com_map.items())-1:
            clusters.append(temp)
    else:
        clusters.append(temp)
        temp = []
        prev_com = cluster
        temp.append(word)
        if index==len(com_map.items())-1:
            clusters.append(temp)

In [4]:
com_map_refined = {}
for ent,cls in com_map.items():
    if len(clusters[cls])<=2:
        continue
    else:
        com_map_refined[ent] = cls

In [6]:
len(com_map_refined)

160

In [53]:
ent_check = "Harvard Business Review"
ck = []
for ent in ent_fv.keys():
    ck.append((ent, 1-cosine(ent_fv[ent], ent_fv[ent_check])))

In [57]:
for ent, score in sorted(ck, key=lambda kv:kv[1], reverse=True)[:30]:
    print (ent, " -> ", score)

Harvard Business Review  ->  1.0
Mckinsey  ->  0.8903322815895081
Forbes  ->  0.8880045413970947
Cio  ->  0.8862068057060242
Engineering  ->  0.8720834255218506
Digital  ->  0.87077397108078
Software Engineering  ->  0.8672767877578735
C-suite  ->  0.8627213835716248
Man-month  ->  0.8586208820343018
Gartner  ->  0.855764627456665
American  ->  0.8541392087936401
Software  ->  0.8452587723731995
Agiles  ->  0.8377225995063782
Agile Software  ->  0.8330069184303284
10x  ->  0.8307514190673828
Kodak  ->  0.8300771713256836
Goldman Sachs  ->  0.8299168944358826
Tech  ->  0.829247772693634
Finance  ->  0.8286100625991821
Agile  ->  0.8278757929801941
North Star  ->  0.8275061249732971
Org  ->  0.8253639936447144
Accelerate  ->  0.8239707350730896
Itil  ->  0.823801577091217
Lean Startup  ->  0.8228476643562317
Nasa  ->  0.8222607970237732
Italian  ->  0.82192063331604
Thoughtworks  ->  0.821498692035675
Epfl  ->  0.820669412612915
British  ->  0.8203485012054443


# testing sentence to entity similarity

In [43]:
import sys
sys.path.append("../")
from extra_preprocess import preprocess_text

#text = "Everybody wants to create applications using the micro service architecture in this video. But rather than reading out a list of architectural Concepts, we will approach that a completely different place by me telling you a story that you really need to know. Let is go back in time to the old days when we did not have anything called micro services. Theyll go to a time when a computer was big enough to fill the room. Mr. Seen pictures of one of these things you would walk up to that computer. Fast forward a little bit to desktop machines again programs recite it on the desktop machine the programmer application contains all the instructions that are needed to execute. So when people would write applications for it their code would be compiled down in the whole thing would be installed on the computer in one go and it will be installed on the same computer think of purely desktop only apps. Okay, something like Microsoft Word or a text editor that you install on your I mean this has historically influence how we write code when we need to write an application. We would start a new code project and we would add functionality into that project need more functionality add more code. And so the size of the code base for any given application keeps increasing over time. So what starts as a small code base might end up turning into a large complex code base over time and people realize this."
text = "Nothing too fancy, but overtime web applications have started becoming better bigger quicker more complicated bigger scale bigger user base bigger everything and today we have web applications that can you know find something from the whole internet in a matter of milliseconds are fine. What cabs are available around you all over the world in milliseconds. These are incredible Feats if you think about it, and they need incredibly complicated code to be Developed and applied and this complexity becomes harder and harder to maintain. We have nice modular architecture on the code side of things is not that enough to handle the complexity during development time? Well with the type of applications we are talking about the complexity needs to be handled not just at the coding side of things. They also need to be handled at the runtime or execution side of things having a single thing that you would apply. Didnt work anymore this way of having a single application is called the monolithic application or monolithic architecture mono means single lithic means Stone single Stone monolithic. This is the Smosh basically What are some of the disadvantages of this monolithic model first the bigger the deployment the more challenging the deployment Let me Give an example. Let is say you want to push a new feature to your big monolithic application. So among all the code comments that you want to deploy is the single quote comment by this new guy. The company has just hired me not so sure about him. He probably does not know much and he still learning but his first code come it ever is still sitting there. Well, you need to test the whole thing before you would apply the whole application you never know which part of the application that come it might have broken. Well, I am exaggerating here, of course, but the fact remains."
text_p = " ".join(preprocess_text(text))

text_pp = " ".join(tp.preprocess(text, stop_words=True))

In [44]:
text_fv = gpt_model.get_text_feats(text)
ck = []
for ent in ent_fv.keys():
    ck.append((ent, 1-cosine(ent_fv[ent], text_fv)))

In [45]:
[i for i,j in sorted(ck, key=lambda kv:kv[1], reverse=True)[:10]]

['Perfect',
 'Harry',
 'Know',
 'Scotsman',
 'Ugh',
 'Open-closed',
 'Greeks',
 'Wait',
 'Retval',
 'Spaghetti Code']

In [46]:
text_fv = gpt_model.get_text_feats(text_p)
ck = []
for ent in ent_fv.keys():
    ck.append((ent, 1-cosine(ent_fv[ent], text_fv)))

In [47]:
[i for i,j in sorted(ck, key=lambda kv:kv[1], reverse=True)[:10]]

['Perfect',
 'Harry',
 'Know',
 'Scotsman',
 'Ugh',
 'Open-closed',
 'Greeks',
 'Wait',
 'Retval',
 'Spaghetti Code']

In [33]:
text_fv = gpt_model.get_text_feats(text_pp)
ck = []
for ent in ent_fv.keys():
    ck.append((ent, 1-cosine(ent_fv[ent], text_fv)))

In [34]:
[i for i,j in sorted(ck, key=lambda kv:kv[1], reverse=True)[:10]]

['Spaghetti Code',
 'Perfect',
 'Know',
 'God',
 'Ugh',
 'Spaghetti',
 'Netscapes',
 'Universe',
 'Smalltalk',
 'Harry']

In [42]:
sent_dict['Perfect']

['the perfect is the enemy of the good.',
 'the perfect is the enemy of the good.',
 "'perfect is the enemy of good.'",
 'perfect we identified our first object role the entity loader ientityloader interface.',
 "'perfect is the enemy of good.'",
 'finally, swift, originally from apple and used on ios, has begun to see some use in web backends as well, with vapor, kitura and perfect.',
 'since then, a variety of other server frameworks have sprung up such as vapor and perfect, both of which seem to be building polished, feature-rich, and well documented tools for constructing a stack built around swift.',
 "'the perfect is the enemy of the good' — voltaire."]